In [3]:
import torch as th
import numpy as np
import sys
sys.path.insert(0, "/home/mint/Dev/SkelAg/DiffSynth-Studio")

from diffsynth.models.wan_video_dit import DiTBlock, rearrange, precompute_freqs_cis_3d, sinusoidal_embedding_1d

n_timesteps = 1000
use_time = [500, 1000]
n_blocks = 30
use_block = [10, 30]
B = 1
D = 1536
F = 1024    # Flattened spatial-temporal dimension (1+T/4 * H/16 * W/16); H, W = 128
feats_dim = (B, F, D)

feats = np.zeros((n_timesteps, n_blocks, B, F, D), dtype=np.uint8)
print(feats.shape)

use_feats = []
for t in use_time:
    for b in use_block:
        # print(f"t: {t}, b: {b}")
        use_feats.append(feats[t-1, b-1])
        # print(use_feats[0].shape)

use_feats = th.tensor(np.stack(use_feats, axis=0))
print("use_feats shape:", use_feats.shape)


(1000, 30, 1, 1024, 1536)
use_feats shape: torch.Size([4, 1, 1024, 1536])


In [ ]:
num_heads = 8
freq_dim =256
dim = D
dit = DiTBlock(
    has_image_input=False,
    dim=D,
    num_heads=num_heads,
    ffn_dim=256,
).cuda()

time_embedding = th.nn.Sequential(
    th.nn.Linear(freq_dim, dim),
    th.nn.SiLU(),
    th.nn.Linear(dim, dim)
).cuda()

time_projection = th.nn.Sequential(
        th.nn.SiLU(), th.nn.Linear(dim, dim * 6)).cuda()

use_feats = use_feats.float().cuda()
x = use_feats
print("Input x shape:", x.shape)
# Patchify
fdim = x.shape[0]

#NOTE: Flatten the spatio-temporal dimensions
x = rearrange(x, 'c b f d -> b (f c) d').contiguous()
print("Rearrange shape: ", x.shape)

timestep = th.tensor([500]).float().cuda()
dit = dit.cuda()
t = time_embedding(sinusoidal_embedding_1d(freq_dim, timestep))
t_mod = time_projection(t).unflatten(1, (6, dim))
head_dim = D // num_heads
freqs = precompute_freqs_cis_3d(D * use_feats.shape[2])
f, h, w = 16, 8, 8  # Example dimensions; adjust as needed
freqs = th.cat([
    freqs[0][:f].view(f, 1, 1, -1).expand(f, h, w, -1),
    freqs[1][:h].view(1, h, 1, -1).expand(f, h, w, -1),
    freqs[2][:w].view(1, 1, w, -1).expand(f, h, w, -1)
], dim=-1).reshape(f * h * w, 1, -1).to(x.device)

print(t_mod.shape)
print(freqs.shape)
out = dit(x, t_mod=t_mod, freqs=freqs, context=None)


Input x shape: torch.Size([4, 1, 1024, 1536])
Rearrange shape:  torch.Size([1, 4096, 1536])


RuntimeError: [enforce fail at alloc_cpu.cpp:124] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 9070970929152 bytes. Error code 12 (Cannot allocate memory)